# Experimentation notebook (testing in this project)

Sources:
- https://www.youtube.com/watch?v=BrsocJb-fAo
- https://github.com/svpino/youtube-rag

In [2]:
# Imports
import os
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser



In [3]:
# Model setup
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
parser = StrOutputParser()

In [4]:
# Template setup

template = """
Respondé las preguntas basandote en el siguiente contexto. Si no podes responder una pregunta. 
Responde: "Necesito más información para responder esta pregunta."

Contexto: {context}

Pregunta: {question}

"""


In [5]:
# Import file 

with open('resumen_tesis.txt', 'r', encoding='utf-8') as file:
    texto = file.read()
texto 

'UNIVERSIDAD DE BUENOS AIRES\nFacultad de Ciencias Exactas y Naturales\nDepartamento de Física\nModelado y calibración en dinámica peatonal:\npropuestas para mejorar las evacuaciones\nen estado de pánico\nTesis presentada para optar al título de Doctor de la Universidad de\nBuenos Aires en el área Ciencias Físicas\nLic. Ignacio Mariano Sticco\nDirector de tesis: Dr. Guillermo Alberto Frank\nConsejero de Estudios: Dr. Ariel Chernomoretz\nLugar de trabajo: Instituto de Física de Buenos Aires (IFIBA)\nBuenos Aires, 2023\nResumen\nEsta tesis estudia la dinámica de multitudes en estado de pánico. Para describir el comportamiento de dichas multitudes se utiliza el modelo de fuerza social de Helbing. En este modelo los individuos se representan como discos blandos que están sometidos a distintos tipos de fuerzas, y la evolución temporal del sistema queda determinada por las ecuaciones de movimiento de la mecánica clásica. La ventaja de este modelo es que permite estudiar fenómenos macroscópic

In [6]:
# Invoke
context = texto

pregunta = "De que trata la tesis?"
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model | parser
chain.invoke({
    "context": context,
    "question": pregunta
})

'La tesis trata sobre el modelado y la calibración en dinámica peatonal, con el objetivo de mejorar las evacuaciones en estado de pánico.'

### Pruebo embedear un texto que yo le paso

In [24]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("resumen_tesis.txt", encoding='utf-8')
text_documents = loader.load()
text_documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

In [25]:
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()


In [26]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [35]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("Si tuvieses que arriesgar, quien pensas que es el autor de la tesis? y por que?")

'El autor de la tesis podría ser Ignacio Mariano Sticco, ya que su nombre aparece al final del texto junto con el título de Licenciado.'